### 3.1 处理缺失数据
#### 1. 缺失值的检测,删除,填充
1. 检测缺失值: 数据结构.isnull() , 返回DataFrame或Series
2. 什么值被识别成缺失值 :   
  1. python的None对象
  2. np.nan对象
3. Series.dropna : 返回不带缺失值的Series  
 DataFrame.dropna(how='all/any' , axis=0/1) : 删除整行/整列, 根据全部缺失或部分缺失
4. 填充缺失值:  
 数据结构.fillna(method='',values='')  
  1. mathod: 填充策略  
    1. ffill : 使用上一个值填充
    2. bfill : 使用下一个值填充
    3. None : 不适用策略,自己指定values指
  2. value : 自定义填充值  
    1. value可以为scalar, 字典, Series, DataFrame
    2. value=字典: (key=columnIdx,value=填充值), 指定每一列的填充值
  


In [141]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

df = pd.DataFrame([[np.nan, 2, np.nan, 0],[3, 4, np.nan, 1], [np.nan, np.nan, np.nan, 5],[np.nan, 3, np.nan, 4]],columns=list('ABCD'))
obj = Series([1,2,3,np.nan,5,6])
print df.isnull()
print obj.isnull()
print "=============[1]============="
# 删除缺失值
print obj.dropna()
print df.dropna(how='all')
print "=============[2]============="
# 填充缺失值
print df.fillna(value={'A':-1,'B':-2,'C':-3})
print df.fillna(method='ffill')

       A      B     C      D
0   True  False  True  False
1  False  False  True  False
2   True   True  True  False
3   True  False  True  False
0    False
1    False
2    False
3     True
4    False
5    False
dtype: bool
=============[1]=============
0    1.0
1    2.0
2    3.0
4    5.0
5    6.0
dtype: float64
     A    B   C  D
0  NaN  2.0 NaN  0
1  3.0  4.0 NaN  1
2  NaN  NaN NaN  5
3  NaN  3.0 NaN  4
=============[2]=============
     A    B    C  D
0 -1.0  2.0 -3.0  0
1  3.0  4.0 -3.0  1
2 -1.0 -2.0 -3.0  5
3 -1.0  3.0 -3.0  4
     A    B   C  D
0  NaN  2.0 NaN  0
1  3.0  4.0 NaN  1
2  3.0  4.0 NaN  5
3  3.0  3.0 NaN  4


### 3.2 数据转换
#### 1. 重复行的处理
1. 检测重复行:   
 DataFrame.duplicated() , 返回Boolean的Series(表示该行是否为重复行)
2. 删除重复行  
  1. DataFrame.drop_duplicates()
  2. DataFrame.drop_duplicates([columnIdx]) : 指定列上若有重复值, 则删除整行  
  [注] : 删除重复行, 默认保留最先出现的行. 可使用keep='last'保留最后出现的行


In [142]:
df = DataFrame({'k1':['one','two']*3+['two'], 'k2':[1,1,2,3,3,4,4]})
df

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [143]:
# 检测重复行
print df.duplicated()
print "=============[1]============="
# 删除重复行
print df.drop_duplicates()
# 删除指定列上有重复值的行
print df.drop_duplicates(['k1'])

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool
=============[1]=============
    k1  k2
0  one   1
1  two   1
2  one   2
3  two   3
4  one   3
5  two   4
    k1  k2
0  one   1
1  two   1


#### 2. 值映射
1. Series.map(dict/lambda func) :  
 将一个Series映射成另一个Series. 若用dict指定, 则是吧Series中的元素匹配key, 转换成对应的value  
 若传入lambda函数 : 把lambda函数应用在Series的每个元素上
 2. DataFrame的值映射  
 由于map方法只能应用在Series上, 而DataFrame能通过apply(f)应用在DataFrame的每列Series上, 所以组合出了applymap(f)方法  
 该方法可以讲func分别应用在DataFrame的每列上

In [144]:
df = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon','Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                                'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
df

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [145]:
# Series的值映射map()
mydict = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
print df['food'].str.lower().map(mydict) # .str提供字符串转换方法
f = lambda x:mydict[x.lower()]
f2 = lambda x:x*2
print df['food'].map(f)
print "=============[1]============="
# DataFrame的值映射
df.applymap(f2)
# DataFrame删除全0行 : 先用applymap把0映射成None,再用dropna(how='all')删除
df2 = DataFrame({'a':[1,2,0],'b':[9,9,0]})
zerofunc = lambda x: None if (x==0) else x 
df2 = df2.applymap(zerofunc)
df2.dropna(how='all')

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object
0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object
=============[1]=============


,a,b
0,1.0,9.0
1,2.0,9.0


#### 3. 值替换
1. 数据结构.fillna可看做值替换的特殊形式: 把NaN替换成指定值  
 Series.map()也是替换数据的子集, replace()提供了更灵活的办法
2. 数据结构.replace(src,dist) : 把src替换成dist  
  1. method参数: 替换策略, 默认为None. 当dist=None时, 按照替换策略替换值.相当于fillna(method=''). 参数选择同样有ffill, backfill  
  2. replace(dict) : 将key作为src, valjue作为dist

In [146]:
 obj = pd.Series([1., -999., 2., -999., -1000., 3.])
print obj
# 将-999替换成NaN
print obj.replace(-999,np.nan)
print obj.replace([-999,-1000],np.nan)
print ("=============[1]============")
# 替换参照dict
replaceDict = {-999:np.nan,-1000:np.nan}
print obj.replace(replaceDict)


0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64
0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64
0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64
=============[1]============
0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64


In [147]:
df.replace(mydict)

,food,ounces
0,pig,4.0
1,pig,3.0
2,pig,12.0
3,Pastrami,6.0
4,cow,7.5
5,Bacon,8.0
6,cow,3.0
7,pig,5.0
8,salmon,6.0


#### 4. index重命名
1. Index.map(func), 使用函数替换索引名称
2. DataFrame.rename(index=func,columns=func) : 替换rowidx和columnIdx的方法

In [148]:
df = pd.DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])
df

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [149]:
df.index = df.index.map(lambda x:x.upper())
df

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [150]:
# Series重命名索引
print df['one'].rename(str.lower)
# DataFrame重命名索引
df.rename(index=str.title,columns=str.lower)

ohio        0
colorado    4
new york    8
Name: one, dtype: int64


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


#### 5. 数据分箱(bin)
1. 数据分箱(bin), 即把Series中的元素, 分配不同箱子(bin)中.  
2. bin的定义 :  
  1. 数组定义: 每个相邻元素形成区间, 作为bin
  2. 标量定义: 表示分成n个bin, 区间范围为最大值和最小值之间分成n份
3. pd.cut(Series, bin) : 切分元素到bin中.  
  1. 返回Categories对象组成的Series. Categories对象有两部分组成, index和bin. bin是表示区间意义的字符串 
  2. 通过分箱后返回的Series[Categroies], 使用value_counts() 查看有多少个bin, 每个bin有多少个元素  
4. pd.qcut(Series, quantiles) : quantile cut按照百分比分箱
  1. 指定分箱上下区间的百分比 (quantiles在[0,1])

In [151]:
ages = Series([20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32],index=list('abcdefghijkl'))
bins = [10,25,35,60,100]
# 1. 切分bin
cats = pd.cut(ages,bins)
print cats
print "================[1]=============="
# 2. 查看bin有哪些
print cats.value_counts()
print "================[2]=============="
# 3. 百分比分箱
pd.qcut(ages,[0,0.1,0.6,0.9,1])

a     (10, 25]
b     (10, 25]
c     (10, 25]
d     (25, 35]
e     (10, 25]
f     (10, 25]
g     (35, 60]
h     (25, 35]
i    (60, 100]
j     (35, 60]
k     (35, 60]
l     (25, 35]
dtype: category
Categories (4, interval[int64]): [(10, 25] < (25, 35] < (35, 60] < (60, 100]]
================[1]==============
(10, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64
================[2]==============


a    (19.999, 21.1]
b      (21.1, 31.6]
c      (21.1, 31.6]
d      (21.1, 31.6]
e    (19.999, 21.1]
f      (21.1, 31.6]
g      (31.6, 44.6]
h      (21.1, 31.6]
i      (44.6, 61.0]
j      (44.6, 61.0]
k      (31.6, 44.6]
l      (31.6, 44.6]
dtype: category
Categories (4, interval[float64]): [(19.999, 21.1] < (21.1, 31.6] < (31.6, 44.6] < (44.6, 61.0]]

#### 6. 检测过滤异常值
有些时候, 某些数据的绝对值过大或过小, 这些值通常被认做异常值(outlier).  
我们构造一个随机生成的DataFrame, 其中某些列的元素绝对值>3, 我们认为, 只要某行记录, 有一列的元素绝对值>3, 就把整行当做异常值.
1. 找出这些异常记录
2. 把这些异常记录中绝对值>3的元素设置为+3/-3

In [1]:
np.random.seed(100)
df = DataFrame(np.random.randn(1000,4))
# 1. 找出异常值 : np.abs(df)>3生成boolean的DataFrame, any(axis=1)找出任何有False的行
print df[(np.abs(df)>3).any(axis=1)]
# 2. 把这些异常记录中绝对值>3的元素设置为+3/-3
# Dataframe之间的赋值操作, 会按照columnIdx和rowIdx对齐
df[np.abs(df)>3] = np.sign(df)*3
print df[(np.abs(df)>3).any(axis=1)] # 此时发现已不存在大于3的元素

NameError: name 'np' is not defined

#### 7. 随机抽样
1. numpy的随机抽样方法  
 numpy.random.permutation(n) : 产生0~n-1的随机排列数组  
 pandas的数据结构.take(随机数组) : 产生原数据结构的随机抽样
2. pandas的随机抽样方法  
 数据结构,sample(n) : 随机抽取数据结构的n个记录  
  1. 可放回随机抽样 : replace=True
  2. numpy随机抽样, 可先使用np.random.randint(0,4,(4,1))产生4个0~4的随机数, 再用数据结构.take(随机数组)进行重复随机抽样

In [153]:
df = pd.DataFrame(np.arange(5*4).reshape(5,4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [154]:
# numpy的随机抽样
sampler = np.random.permutation(5)
print df.take(sampler)
print "================[1]=============="
# pandas随机抽样
print df.sample(4)
print "================[2]=============="
# 可放回抽样
print df.sample(4,replace=True)

    0   1   2   3
0   0   1   2   3
4  16  17  18  19
2   8   9  10  11
3  12  13  14  15
1   4   5   6   7
================[1]==============
    0   1   2   3
1   4   5   6   7
2   8   9  10  11
4  16  17  18  19
0   0   1   2   3
================[2]==============
   0  1   2   3
0  0  1   2   3
2  8  9  10  11
0  0  1   2   3
1  4  5   6   7


#### 9. 类型的哑变量化
1. pd.get_dummies(数据结构) : 把数据结构中的类型变量取值变成columnIdx, 新产生的columnIdx取值为0或1  
2. 一个比较有用的组合 : pd.cut()先把列Series分箱到bin, 再把bin提升成哑变量(pd.get_dummies)

In [212]:
np.random.seed = 10
# 给定一个DataFrame, 把这个DataFrame的a列提升成哑变量
# 将DataFrame的a列分箱后, 变成箱子的哑变量, 和去除a列后的DataFrame拼接成新的DataFrame
df = DataFrame(np.random.randint(0,5,(4,5)),columns=list('abcde'))
cutSeries = pd.cut(df['a'],3)
adummiesDF = pd.get_dummies(cutSeries)
# DataFrame默认drop行, drop列需要指定axis=1
dropDF = df.drop('a',axis=1)
# pandas不允许CategoricalIndex连接其他类型Index的DataFrame, 但反过来可以, 所以只能pd.concat(dropDF,adummiesDF)
pd.concat([dropDF,adummiesDF],axis=1)

,b,c,d,e,"(0.999, 1.333]","(1.333, 1.667]","(1.667, 2.0]"
0,0,2,4,1,0,0,1
1,2,1,2,4,0,0,1
2,4,4,3,0,0,0,1
3,3,0,1,4,1,0,0


####  10. pandas对字符串类型的Series处理
1. 对Series的每一个字符串元素, 可通过Series.map(func)方法映射其元素. 但当Series中的元素为NaN时, map方法会报错  
 为解决这个问题, pandas封装了一些Series的字符串操作方法, 在Series.str对象中.  
  1. 查询每个元素字符串是否包含str1: Series.str.contains(str1)

In [213]:
obj = Series({'Dave': 'dave@google.com', 
         'Steve': 'steve@gmail.com',
         'Rob': 'rob@gmail.com', 
         'Wes': np.nan})
obj

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [214]:
obj.str.contains('dave')

Dave      True
Rob      False
Steve    False
Wes        NaN
dtype: object

In [215]:
# 正则匹配
import re
pattern = '([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'
obj.str.findall(pattern,flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [216]:
obj.str.match(pattern,flags=re.IGNORECASE)
obj.str[1]
# obj.str.get(1)

Dave       a
Rob        o
Steve      t
Wes      NaN
dtype: object